# Heroes of Pymoli

In [29]:
import numpy as np
import pandas as pd

file1 = "raw_data/purchase_data.json"

purchase_data = pd.read_json(file1, orient="records")


In [30]:
purchase_data.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


# Player Count

In [31]:
player_demographics = purchase_data.loc[:, ["Gender", "SN", "Age"]]
player_demographics.head()

,Gender,SN,Age
0,Male,Aelalis34,38
1,Male,Eolo46,21
2,Male,Assastnya25,34
3,Male,Pheusrical25,21
4,Male,Aela59,23


In [32]:
player_demographics = player_demographics.drop_duplicates()
num_players = player_demographics.count()[0]
num_players

573

In [33]:
pd.DataFrame({"Total Players": [num_players]})

,Total Players
0,573


# Purchasing Analysis(Total)

In [43]:
#Basic Calculations
average_item_price = purchase_data["Price"].mean()
total_purchase_value = purchase_data["Price"].sum()
purchase_count = purchase_data["Price"].count()
item_count = len(purchase_data["Item ID"].unique())

# Create DataFrame to hold results
summary_table = pd.DataFrame({"Number of Unique Items": item_count,
                               "Total Revenue": [total_purchase_value],
                               "Number of Purchases": [purchase_count],
                               "Average Purchase Price": [average_item_price]})

# Data Munging
summary_table = summary_table.round(2)
summary_table ["Average Price"] = summary_table["Average Purchase Price"].map("${:,.2f}".format)
summary_table ["Total Revenue"] = summary_table["Total Revenue"].map("${:,.2f}".format)
summary_table = summary_table.loc[:, ["Number of Unique Items", "Average Purchase Price", "Number of Purchases", "Total Revenue"]]

summary_table

,Number of Unique Items,Average Purchase Price,Number of Purchases,Total Revenue
0,183,2.93,780,"$2,286.33"


# Gender Demographics

In [47]:
# Basic Calculations
gender_demographics_totals = player_demographics["Gender"].value_counts()
gender_demographics_percents = gender_demographics_totals / num_players * 100

# Data Cleanup
gender_demographics = pd.DataFrame({"Total Count": gender_demographics_totals,
                                    "Percentage of Players": gender_demographics_percents})

gender_demographics = gender_demographics.round(2)

#Data Gender_demographics
gender_demographics

,Percentage of Players,Total Count
Male,81.15,465
Female,17.45,100
Other / Non-Disclosed,1.40,8


In [48]:
gender_demographics_percents

Male                     81.151832
Female                   17.452007
Other / Non-Disclosed     1.396161
Name: Gender, dtype: float64

# Purchasing Analysis (Gender)

In [61]:
# Basic Calculations
gender_purchase_total = purchase_data.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Value")
gender_average = purchase_data.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Value")
gender_counts = purchase_data.groupby(["Gender"]).count()["Price"].rename("Purchase Count")


# Calculate normalized Purchasing
normalized_total = gender_purchase_total / gender_demographics["Total Count"]

# Cleanup
gender_data = pd.DataFrame({"Normalized Total": normalized_total, 
                            "Purchase Count": gender_counts, 
                            "Total Purchase Value": gender_purchase_total, 
                            "Average Purchase Value": gender_average})


#Display
gender_data

,Average Purchase Value,Normalized Total,Purchase Count,Total Purchase Value
Gender,,,,
Female,2.815515,3.829100,136,382.91
Male,2.950521,4.016516,633,1867.68
Other / Non-Disclosed,3.249091,4.467500,11,35.74


# Age Demographics

In [66]:
age_bins = [0, 9.90, 14.90, 19.90, 24.9, 29.9, 34.90, 39.90, 9999999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", ">40"]

#Use Cut to categorize players
player_demographics["Age Ranges"] = pd.cut(player_demographics["Age"], age_bins, labels=group_names)
player_demographics.head()

,Gender,SN,Age,Age Ranges
0,Male,Aelalis34,38,35-39
1,Male,Eolo46,21,20-24
2,Male,Assastnya25,34,30-34
3,Male,Pheusrical25,21,20-24
4,Male,Aela59,23,20-24


In [70]:
age_demographics_totals = player_demographics["Age Ranges"].value_counts()
age_demographics_percents = age_demographics_totals / num_players * 100

age_demographics = pd.DataFrame({"Total Count": age_demographics_totals, "Percent of Players": age_demographics_percents})
age_demographics = age_demographics.sort_index()

In [71]:
age_demographics

,Percent of Players,Total Count
10-14,4.013962,23
15-19,17.452007,100
20-24,45.200698,259
25-29,15.183246,87
30-34,8.202443,47
35-39,4.712042,27
<10,3.315881,19
>40,1.919721,11


# Purchasing Analysis (Age)

# Top Spenders

In [75]:
#Basic Calculations
user_total = purchase_data.groupby(["SN"]).sum()["Price"].rename("Total Purchase Amount")
user_average = purchase_data.groupby(["SN"]).mean()["Price"].rename("Average Purchase Price")
user_count = purchase_data.groupby(["SN"]).count()["Price"].rename("Purchase Count")

user_data = pd.DataFrame({"Total Purchase Amount": user_total,
                          "Average Purchase Price": user_average,
                          "Purchase Count": user_count})

# Convert to Data Frame
user_data.sort_values("Total Purchase Amount", ascending=False).head(5)


,Average Purchase Price,Purchase Count,Total Purchase Amount
SN,,,
Undirrala66,3.412000,5,17.06
Saedue76,3.390000,4,13.56
Mindimnya67,3.185000,4,12.74
Haellysu29,4.243333,3,12.73
Eoda93,3.860000,3,11.58


# Most Popular Items

In [76]:
user_total = purchase_data.groupby(["Item ID", "Item Name"]).sum()["Price"]
user_total.head()

Item ID  Item Name         
0        Splinter              1.82
1        Crucifer              9.12
2        Verdict               3.40
3        Phantomlight          1.79
4        Bloodlord's Fetish    2.28
Name: Price, dtype: float64